## **Atividade 2 — Aprendizado não Supervisionado**

Nesta atividade, você deve utilizar **métodos de aprendizado não supervisionado** vistos em sala de aula, como **k-means** ou **redes SOM**, para **identificar padrões nos dados de engajamento**.

O objetivo é compreender **quais tópicos, padrões ou grupos de postagens** estão mais associados a alto ou baixo engajamento.

In [ ]:
!gdown 19Y-NdpOTCfmtlM66FqodAtiZMs8lGQTq

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

df = pd.read_pickle('df_social_data_train.pkl')
df

## Pré Processando os Dados e Classificando-os em clusters com K-Means

In [ ]:
# pre-processar e extrair caracteristicas dos dados para construir a tabela atributo-valor
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

df = df.dropna()

In [ ]:
df['features'] = list(model.encode(df['content'].tolist(), show_progress_bar=True))

df

In [ ]:
df[['content','features']].head(10)

Separamos o conjunto de dados entre:
- Posts com engajamento low
- Posts com engajamento high

In [ ]:
df_low = df[df['engagement'] == 'low']
df_high = df[df['engagement'] == 'high']
features_low = np.array(df_low.features.to_list())
features_high = np.array(df_high.features.to_list())

KMeans com seu valor padrão de clusteres: 8

In [ ]:
kmeans_low = KMeans(random_state=0, n_init="auto").fit(features_low)
kmeans_high = KMeans(random_state=0, n_init="auto").fit(features_high)
df_low['cluster'] = kmeans_low.labels_
df_high['cluster'] = kmeans_high.labels_


In [ ]:
df_low

Plot dos clusteres para visualização

In [ ]:
import umap

y_low = np.array(df_low.cluster.to_list())
y_high = np.array(df_high.cluster.to_list())

features2D_low = umap.UMAP().fit_transform(features_low, y=y_low)
features2D_high = umap.UMAP().fit_transform(features_high, y=y_high)

In [ ]:
sns.set(style='white', context='poster')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))  # 1 row, 2 columns

# Plot for low cluster data on the first subplot
axes[0].scatter(features2D_low[:, 0], features2D_low[:, 1], s=0.1,
                c=df_low.cluster.to_list(), cmap='Spectral', alpha=1.0)
axes[0].set_title('Low Cluster')

# Plot for high cluster data on the second subplot
axes[1].scatter(features2D_high[:, 0], features2D_high[:, 1], s=0.1,
                c=df_high.cluster.to_list(), cmap='Spectral', alpha=1.0)
axes[1].set_title('High Cluster')

plt.tight_layout()
plt.show()


## Instanciação de LLM Ollama 3.2

O reconhecimentos dos temas comuns entre os clusteres será feito utilizando uma LLM. A IA generativa deverá inferir:
- O Tema Geral dos posts agregados em um mesmo cluster
- Palavras chaves que descrevem esse cluster
- Quais as características de posts dentro deste cluster
- Criar um "post" de exemplo que ilustre esse cluster

In [ ]:
!pip install openai==0.28

In [ ]:
from getpass import getpass
import openai

def llm_local():
  openai.api_base = "http://localhost:11434/v1"
  openai.api_key = "ollama"  # Requerido por algumas bibliotecas mesmo que não usado

In [ ]:
import json
import openai

def llm_task(model, system, prompt):
  response = openai.ChatCompletion.create(
      model=model,
      messages = [
            {
              "role": "system",
              "content": system
            },
            {
              "role": "user",
              "content": prompt
            },
          ]

  )
  s = response['choices'][0]['message']['content'].strip()

  return s

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!pip install ollama
!nohup ollama serve &

In [ ]:
# baixando uma LLM (llama3.2)
!ollama pull llama3.2

In [ ]:
llm_local()

## Systems Roles

Para as chamadas à LLM Ollama, haverão três sistemas para obter as informações desejadas. O Primeiro terá o papel de identificar as características mencionadas anteriormete. O Segundo será responsável por estruturar a resposta do primeiro em um formato JSON. O terceiro será usado para verificar se a resposta do segundo está no formato correto de JSON.

In [ ]:
system = '''
You are a social media post analyst.
Given a list of posts, you must:
1) Identify ONLY ONE general theme of the posts. DO NOT GIVE ME MORE THAN ONE THEME.
2) Identify the most relevant topics in the posts. Each topic should consist of 2 or 3 words.
3) Identify characteristics of how those posts are written, if it is personal or not, if it uses emojis, etc.
4) Summarize the posts into a single one

Structure your response as follows:
  - General Theme: A text that clearly summarizes the general theme of the received posts.
  - Relevant Topics: A list of relevant topics you identified in the received posts.
  - Characteristics: A text that clearly summarizes the characteristics of the posts.
  - Summary of posts: A text that summarizes the posts.

Write the output in Portuguese and ONLY in the structure I provided.
Do not return anything beyond that.
'''


In [ ]:
system2 = '''
Você é um formatador de análise de posts em redes sociais. Você será utilizado para formatar uma resposta anterior em um JSON.
Você receperá um texto em inglês, interprete-o e depois formate em um JSON em português.
A saída da análise deve ser estruturada em JSON.

Exemplo de Estrutura do JSON:

```json
{
  "tema_geral": "texto do tema geral",
  "topicos_relevantes": ["topico A","topico B","topico C"],
  "caracteristicas" : "caracteristicas do post",
  "resumo_posts": "resumir posts",
}
```

A resposta deve ser em português e APENAS EM JSON.
Não retorne nada além do JSON. Não precisa me responder, apenas envie o JSON.
'''

In [ ]:
system3 = '''
Você é uma ferramenta que corrige eventuais erros de JSON.
SE ESTIVER CORRETO APENAS ME RETORNE O JSON.

Exemplo de Estrutura do JSON correto:


{
  "tema_geral": "texto do tema geral",
  "topicos_relevantes": ["topico A","topico B","topico C"],
  "caracteristicas" : "caracteristicas do post",
  "resumo_posts": "resumir posts",
}

A resposta deve ser em português e APENAS EM JSON.
Não retorne nada além do JSON. Não precisa me responder, apenas envie o JSON.
'''

In [ ]:
df_low.sample(10)

chamadas à LLM para identificação das características

In [ ]:
import json

L = []
low_clusters = {}
high_clusters = {}

def identify_clusters(df, cluster_dict):
  for cluster in range(0,8):
    print("\n\nGerando analise do Cluster", cluster)
    prompt = ''
    df_temp = df[df.cluster == cluster].sample(30)
    for index,row in df_temp.iterrows():
      prompt += f'#Post: {row.content} \n\n\n'
    #print(prompt)

    resposta = llm_task("llama3.2",system,prompt)
    print(resposta)
    resposta2 = llm_task("llama3.2",system2,resposta)
    json_str = resposta2.replace("```json","").replace("```","")
    obj = json.loads(json_str)
    print(obj)

    df_temp['tema_geral'] = obj['tema_geral']
    df_temp['topicos_relevantes'] = str(obj.get('topicos_relevantes', []))
    df_temp['caracteristicas'] = str(obj.get('caracteristicas', []))
    df_temp['resumo_posts'] = str(obj.get('resumo_posts', []))

    L.append(df_temp)
    cluster_dict[cluster] = obj
    print("======")



identify_clusters(df_low, low_clusters)
identify_clusters(df_high, high_clusters)

In [ ]:
df_clusters = pd.concat(L)
df_clusters

## Print final dos Resultados Obtidos

Low clusteres

In [ ]:
for key, value in low_clusters.items():
  print("Cluster: ", key)
  print("Tema Geral: ", value['tema_geral'])
  print("Tópicos Relevantes: ", value['topicos_relevantes'])
  try:
    print("Caracteristicas: ", value['caracteristicas'])
  except:
    print("Caracteristicas: ")
  print("Resumo Posts: ", value['resumo_posts'])
  print("\n")

High clusteres

In [ ]:
for key, value in high_clusters.items():
  print("Cluster: ", key)
  print("Tema Geral: ", value['tema_geral'])
  print("Tópicos Relevantes: ", value['topicos_relevantes'])
  try:
    print("Caracteristicas: ", value['caracteristicas'])
  except:
    print("Caracteristicas: ")
  print("Resumo Posts: ", value['resumo_posts'])
  print("\n")

## **Atividade 3 — Desenvolvimento de um Agente Criativo para Geração de Postagens Engajadas**

Esta é uma atividade **aberta à criatividade dos alunos**. O desafio consiste em desenvolver um **agente inteligente** que utilize **modelos de aprendizado profundo ou inteligência artificial generativa** para **auxiliar na criação de postagens com maior potencial de engajamento**.

Esse agente pode explorar:

* Análise de características de postagens com alto engajamento
* Reescrita ou sugestão de novos textos com base em um post inicial
* Combinação de diferentes abordagens estudadas ao longo da disciplina



# Solução:
O agente proposto possuirá uma ferramenta de melhora de Posts, visando um aumento no possível engajamento.
Para isso, a ferramenta de MelhorarPost fará uma chamada à LLM Ollama, possuindo como contexto, as características identificadas no exercício anterior.
A LLM tentará:
- Inferir a qual cluster esse Post melhor se assimilaria
- Modificar o Post Original com base nas anotações de engajamento 'High' previstas para aquele cluster
- Retornar o Post Final com as mudanças efetuadas 

In [ ]:
!pip install -q langchain langchain-community

In [ ]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3.2")

In [ ]:
# testando a LLM
from langchain_core.messages import HumanMessage

resposta = llm([HumanMessage(content="Olá LLM")])
print(resposta.content)

System que da o contexto do sistema (nesse caso é apenas uma parte do prompt)

In [ ]:
system4 = '''
Você é um modificador de posts em redes sociais.
Você receberá um post a ser classificado entre os clusters e melhorado.
As suas mudanças deverão ser baseadas nas características dos Clusteres.

Os clusteres que você terá disponíveis para ver as características serão:
'''

for key, value in high_clusters.items():
  system4 += f"#Cluster: {key} \n"
  system4 += f"#Tema Geral: {value['tema_geral']} \n"
  system4 += f"#Tópicos Relevantes: {value['topicos_relevantes']} \n"
  try:
    system4 += f"#Caracteristicas: {value['caracteristicas']} \n"
  except:
    continue

system4 += '''

⚠️ SUA RESPOSTA DEVE CONTER O POST FINAL MELHORADO. NÃO INCLUA ANÁLISES OU PENSAMENTOS, APENAS O POST FINAL.

RETORNE APENAS O POST FINAL.
RETORNE APENAS O POST FINAL.
RETORNE APENAS O POST FINAL.
RETORNE APENAS O POST FINAL.
RETORNE APENAS O POST FINAL.
RETORNE APENAS O POST FINAL.
RETORNE APENAS O POST FINAL.

'''

Criação da tool de melhorar post

In [ ]:
from langchain.tools import Tool

# Função auxiliar para exibir resultados formatados

# Função principal da tool
def melhorar_post_simples(post: str) -> str:
    prompt = system4 + f"\nPost: {post}"
    resposta = llm.invoke([HumanMessage(content=prompt)])
    return resposta.content.strip()

# Criar a tool para LangChain
tool_melhorarPost = Tool(
    name="Melhorar_Post",
    func=melhorar_post_simples,
    description=(
        "Melhorar o post de uma rede social."
        "Use esta ferramenta quando a pergunta envolver melhorar Posts que serão usados para rede sociais. Retornar apenas o conteúdo do post, nada de comentários"
    ),
    return_direct=True
)


Criação do agente utilizando a tool

In [ ]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

agent = initialize_agent(
    tools=[tool_melhorarPost],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

Testes para validar a tool

In [ ]:
query = "Pode melhorar esse post para mim? 'Hoje acordei cedo e trabalhei bastante. Produtividade é tudo!'"
response = agent.run(query)
print(response)


In [ ]:
query = "Pode melhorar esse post para mim? 'Às vezes tudo o que precisamos é de um café forte e foco total.'"
response = agent.run(query)
print(response)